Metric Ideas:
- Youtube “Interaction’
    - Function of likes, dislikes, comments (possibly take into account comments/ratings_disabled)
    - Measures how much interaction users had with the content
- Views
- Category (get mapping through XX_category_id.json files)

- Time
    - Trending date
        - Talk about this (our data is already “trending data”, so we’d be looking at what trends most and least amongst already trending videos, not all videos) —> do we want more data?

Northern Meteorological Seasons
- Spring
    - March 1 to May 31
- Summer
    - June 1 to August 31
- Fall
    - September 1 to November 30
- Winter
    - December 1 to February 28 (February 29 on a leap year)

In [10]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib

# Load the US dataset into a DataFrame
df = pd.read_csv('./../youtube-new/USvideos.csv')

print("This dataset contains " + str(df.shape[0]) + " entries and " + str(df.shape[1]) + " features.")

The data contains: 40949 entries and 16 features.


In [13]:
print("The first video was published " + str(df['publish_time'].min()) + " and the last video was published " + str(df['publish_time'].max()))

The first video was published 2006-07-23T08:24:11.000Z and the last video was published 2018-06-14T01:31:53.000Z


In [27]:
json = pd.read_json('./../youtube-new/US_category_id.json')
# Get video category id and title. id: video_categories[0], title: video_categories[1]
video_categories = [[item['id'], item['snippet']['title']] for item in json['items']]

In [86]:
# Split the data into the four seasons
spring = [entry[1] for entry in df.iterrows() if entry[1]['trending_date'][-2:] in ['03', '04', '05']]
summer = [entry[1] for entry in df.iterrows() if entry[1]['trending_date'][-2:] in ['06', '07', '08']]
fall = [entry[1] for entry in df.iterrows() if entry[1]['trending_date'][-2:] in ['09', '10', '11']]
winter = [entry[1] for entry in df.iterrows() if entry[1]['trending_date'][-2:] in ['12', '01', '02']]

[video_id                                                        tpa4kp4lK60
 trending_date                                                      18.01.05
 title                                                 I have a brain tumor.
 channel_title                                                 Simone Giertz
 category_id                                                              28
 publish_time                                       2018-04-30T15:28:50.000Z
 tags                                                                 [none]
 views                                                               1270699
 likes                                                                121206
 dislikes                                                               1664
 comment_count                                                         25892
 thumbnail_link               https://i.ytimg.com/vi/tpa4kp4lK60/default.jpg
 comments_disabled                                                     False